### poc and wip for multivar regression

# Imports and setup

General imports

In [ ]:
import pandas as pd

OpenGrid-specific imports

In [ ]:
from opengrid.library import houseprint
from opengrid import config
from opengrid.library import regression

c = config.Config()

Plotting settings

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = 16,8

# Load Data

We are going to use gas consumption data and weather data. Because we don't want to overload the weather API, we will only use 1 location (Ukkel).

First, let's define the start and end date of our experiment. Let's take 1 year worth of data, starting with last month.

In [ ]:
end = pd.Timestamp.today().replace(day=1).normalize() + pd.Timedelta(days=1)
start = (end.replace(year=end.year-1)) - pd.Timedelta(days=2)

start = start.tz_localize('Europe/Brussels')
end = end.tz_localize('Europe/Brussels')
print(start, end)

If we take 2016 data for identification, we can use the model to backcast the consumption of 2017. 

In [ ]:
start = pd.Timestamp('2015-12-31', tz='Europe/Brussels')
end = pd.Timestamp('2017-01-02', tz='Europe/Brussels')

## Gas Data

In [ ]:
# Load the Houseprint, and sync all data
hp = houseprint.Houseprint()
#hp = houseprint.load_houseprint_from_file('cache_hp.hp')
hp.init_tmpo()

In [ ]:
hp.sync_tmpos()

In [ ]:
#hp.save('cache_hp.hp')

In [ ]:
def gas_data_generator():
    # Preferred method: as accurate as 3, and faster
    # Daily approach, obtain fully correct daily data.
    # To be aggregated to monthly or weekly or ...
    
    for gas_sensor in hp.get_sensors(sensortype='gas'):
        df = gas_sensor.get_data(head=start, tail=end, resample='day', unit='kWh', diff=False, tz='Europe/Brussels')
        df = df.diff().shift(-1).dropna()
        if df.empty:
            continue
        yield df

def elec_data_generator():
    # Preferred method: as accurate as 3, and faster
    # Daily approach, obtain fully correct daily data.
    # To be aggregated to monthly or weekly or ...
    
    for sensor in hp.get_sensors(sensortype='electricity'):
        df = sensor.get_data(head=start, tail=end, resample='day', unit='kWh', diff=False, tz='Europe/Brussels')
        df = df.diff().shift(-1).dropna()
        if df.empty:
            continue
        yield df

Let's have a peek

In [ ]:
gas_data = gas_data_generator()
elec_data = elec_data_generator()

In [ ]:
peek = next(gas_data)
peek.plot()

## Weather Data

Run this block to download the weather data and save it to a pickle. This is a large request, and you can only do 2 or 3 of these per day before your credit with Forecast.io runs out!

To get the data run the cell below

In [ ]:
from opengrid.library import forecastwrapper
weather = forecastwrapper.Weather(location=(50.8024, 4.3407), start=start, end=end - pd.Timedelta(days=1))
irradiances=[
    (0, 90), # north vertical
    (90, 90), # east vertical
    (180, 90), # south vertical
    (270, 90), # west vertical
]
orientations = [0, 90, 180, 270]
weather_data = weather.days(irradiances=irradiances, 
                            wind_orients=orientations, 
                            heating_base_temperatures=[0, 6, 8 ,10, 12, 14, 16, 18]).dropna(axis=1)
weather_data.drop(['icon', 'summary'], axis=1, inplace=True)
weather_data = weather_data.applymap(float)

In [ ]:
weather_data.head()

In [ ]:
weather_data.columns

# Put data together

I wrote a generator that uses our previously defined generator so you can generate while you generate.

In [ ]:
def analysis_data_generator():
    gas_data = gas_data_generator()
    for gas_df in gas_data:
        gas_df.name='gas'
        df = pd.concat([gas_df, weather_data], axis=1).dropna()
        df = df.tz_convert('Europe/Brussels')
        yield df

Let's have another peek

In [ ]:
analysis_data = analysis_data_generator()

In [ ]:
peek = next(analysis_data)
peek = peek.resample(rule='MS').sum()

fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.plot_date(peek.index, peek['gas'], '-', color='grey', lw=8, label='gas')
for column in peek.columns[1:]:
    if 'heatingDegreeDays' in column:
        ax2.plot_date(peek.index, peek[column], '-', label=column)
plt.legend()

# Run Regression Analysis


We run the analysis on monthly and weekly basis.

In [ ]:
analysis_data = analysis_data_generator()
mrs_month = []
mrs_week = []
for data in analysis_data: 
    data.drop(['moonPhase', 'windBearing', 'temperatureMaxTime', 'temperatureMinTime',
               'apparentTemperatureMaxTime', 'apparentTemperatureMinTime', 'uvIndexTime', 
               'sunsetTime', 'sunriseTime', 'precipProbability'], 
              axis=1, inplace=True)
    df = data.resample(rule='MS').sum()
    mrs_month.append(regression.MVLinReg(df, 'gas', p_max=0.05)) 
    mrs_month[-1].plot()
    
    df = data.resample(rule='W').sum()
    mrs_week.append(regression.MVLinReg(df, 'gas', p_max=0.05)) 
    mrs_week[-1].plot()